In [1]:
import pdfplumber
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pdf = pdfplumber.open("Magnolia-Warehouse.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2g/fbs070qj3qd3kw4ntm252trc0000gp/T/ipykernel_80681/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
def get_all_data(parcel_number, agree):
    driver.get('https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop')
    if agree:
        driver.find_element(By.ID, "btAgree").click()
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    # print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    address_data = driver.find_element(By.CLASS_NAME, "DataletHeaderBottom")
    parcel_id = parcel_id_dirty.split(": ")[1]
    sales_data = []
    try:
        sales_details = driver.find_element(By.ID, "Sales Summary")
        rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
        for row in rows:
            all_data = row.find_elements(By.TAG_NAME, "td")
            if len(all_data) < 5:
                print("insufficient data")
                continue
            else:
                data = {}
                date = all_data[0].text
                book = all_data[1].text
                page = all_data[2].text
                price = all_data[3].text
                grantee = all_data[-1].text
                address = address_data[-1].text
                data["date"] = date
                data["book_number"] = book
                data["page"] = page
                data["sale_price"] = price
                data["grantee_name"] = grantee
                data["parcel_id_new"] = parcel_id
#                 data["address"] = address
                sales_data.append(data)
    except:
        data = {}
        data["date"] = "Not found"
        data["book_number"] = "Not found"
        data["page"] = "Not found"
        data["sale_price"] = "Not found"
        data["grantee_name"] = "Not found"
        data["parcel_id_new"] = "Not found"
        data["address"] = "Not found"
        sales_data.append(data)
    return(sales_data)

In [5]:
all_sales_data = []

In [6]:
agree = True
for index, each_parcel_dict in enumerate(cleaned_list_of_dicts[:5]):
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
    print(index, "done")

0 done
1 done
2 done
3 done
4 done


In [7]:
all_sales_data

[{'date': 'Not found',
  'book_number': 'Not found',
  'page': 'Not found',
  'sale_price': 'Not found',
  'grantee_name': 'Not found',
  'parcel_id_new': 'Not found',
  'address': 'Not found'},
 {'date': 'Not found',
  'book_number': 'Not found',
  'page': 'Not found',
  'sale_price': 'Not found',
  'grantee_name': 'Not found',
  'parcel_id_new': 'Not found',
  'address': 'Not found'},
 {'date': 'Not found',
  'book_number': 'Not found',
  'page': 'Not found',
  'sale_price': 'Not found',
  'grantee_name': 'Not found',
  'parcel_id_new': 'Not found',
  'address': 'Not found'},
 {'date': 'Not found',
  'book_number': 'Not found',
  'page': 'Not found',
  'sale_price': 'Not found',
  'grantee_name': 'Not found',
  'parcel_id_new': 'Not found',
  'address': 'Not found'},
 {'date': 'Not found',
  'book_number': 'Not found',
  'page': 'Not found',
  'sale_price': 'Not found',
  'grantee_name': 'Not found',
  'parcel_id_new': 'Not found',
  'address': 'Not found'}]